In [768]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import random
from sklearn.preprocessing import StandardScaler
import plotly.express as px


In [769]:
final_df = pd.read_csv('../data/movie_final_dataset.csv')
final_df

,Name,Genres,Budget(USD)_Inflated,Domestic(USD)_Inflated,Domestic_Percentage,Foreign(USD)_Inflated,Foreign_Percentage,Worldwide(USD)_Inflated,Runtime(mins),Rating,...,period piece,animation,teen,film adaptation,musical,history,coming of age,sports,war,Worldwide_profit
0,10 cloverfield lane,"drama, thriller, horror, sci-fi",6.076746e+06,8.760602e+07,0.654010,4.634613e+07,0.345990,1.339521e+08,103,pg-13,...,0,0,0,0,0,0,0,0,0,22.043400
1,"10,000 bc","drama, thriller, action, adventure, romance fi...",1.441563e+08,1.301309e+08,0.351333,2.402606e+08,0.648667,3.703914e+08,109,pg-13,...,0,0,0,0,0,0,0,0,0,2.569373
2,12 rounds,"thriller, action, crime",2.726275e+07,1.667757e+07,0.708013,6.877890e+06,0.291987,2.355546e+07,108,pg-13,...,0,0,0,0,0,0,0,0,0,0.864016
3,12 strong,"drama, action, history, war",3.979050e+07,5.209113e+07,0.644274,2.876133e+07,0.355726,8.085246e+07,130,r,...,0,0,0,0,0,1,0,0,1,2.031954
4,12 years a slave,"drama, biography, history",2.539260e+07,7.195247e+07,0.301875,1.663993e+08,0.698125,2.383517e+08,134,r,...,0,0,0,0,0,1,0,0,0,9.386660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,zoolander,comedy,4.851315e+07,7.826600e+07,0.743197,2.704388e+07,0.256803,1.053099e+08,90,pg-13,...,0,0,0,0,0,0,0,0,0,2.170749
2685,zoolander 2,"comedy, action, adventure, romance film, mystery",6.076746e+07,3.506124e+07,0.508592,3.387664e+07,0.491408,6.893788e+07,101,pg-13,...,0,0,0,0,0,0,0,0,0,1.134454
2686,zoom,"action, adventure, fantasy, sci-fi",5.014200e+07,1.717625e+07,0.958658,7.407176e+05,0.041342,1.791697e+07,93,pg,...,0,0,0,0,0,0,0,0,0,0.357325
2687,zootopia,"comedy, action, adventure, mystery, crime, fam...",1.823024e+08,4.147601e+08,0.333340,8.294952e+08,0.666660,1.244255e+09,108,pg,...,0,1,0,0,0,0,0,0,0,6.825228


In [770]:
final_df.columns

Index(['Name', 'Genres', 'Budget(USD)_Inflated', 'Domestic(USD)_Inflated',
       'Domestic_Percentage', 'Foreign(USD)_Inflated', 'Foreign_Percentage',
       'Worldwide(USD)_Inflated', 'Runtime(mins)', 'Rating', 'Audience_Score',
       'Critics_Score', 'plot_summary', 'Month', 'Day', 'Year',
       'Foreign_higher', 'emotion', 'emotion_score', 'drama', 'comedy',
       'thriller', 'action', 'adventure', 'romance film', 'horror', 'fantasy',
       'mystery', 'crime', 'indie', 'biography', 'family', 'sci-fi',
       'period piece', 'animation', 'teen', 'film adaptation', 'musical',
       'history', 'coming of age', 'sports', 'war', 'Worldwide_profit'],
      dtype='object')

## Pre-processing for OLS 

In [771]:
# Dropping columns deemed irrelevant for the model
df_ols = final_df.drop(columns=[ "Genres", "Day","Name", "plot_summary", "Worldwide(USD)_Inflated", "Domestic(USD)_Inflated", "Foreign(USD)_Inflated", "Worldwide(USD)_Inflated", "Foreign_higher", "emotion_score", "Worldwide_profit"])

In [772]:
df_ols.head(5)

,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Rating,Audience_Score,Critics_Score,Month,Year,emotion,...,sci-fi,period piece,animation,teen,film adaptation,musical,history,coming of age,sports,war
0,6.076746e+06,0.654010,0.345990,103,pg-13,0.79,0.91,Mar,2016,fear,...,1,0,0,0,0,0,0,0,0,0
1,1.441563e+08,0.351333,0.648667,109,pg-13,0.37,0.10,Mar,2008,fear,...,0,0,0,0,0,0,0,0,0,0
2,2.726275e+07,0.708013,0.291987,108,pg-13,0.45,0.31,Mar,2009,anger,...,0,0,0,0,0,0,0,0,0,0
3,3.979050e+07,0.644274,0.355726,130,r,0.62,0.50,Jan,2018,fear,...,0,0,0,0,0,0,1,0,0,1
4,2.539260e+07,0.301875,0.698125,134,r,0.90,0.95,Oct,2013,sadness,...,0,0,0,0,0,0,1,0,0,0


Standardizing columns

In [773]:
# Standardizing 

columns_to_scale = ['Budget(USD)_Inflated', 'Domestic_Percentage', 'Foreign_Percentage', 'Runtime(mins)', 'Audience_Score', 'Critics_Score']
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_ols[columns_to_scale])
df_ols[columns_to_scale] = scaled_data


One hot encoding columns

In [774]:
# one-hot encoding rating column, dropping the first column in order to avoid introducing multicollinearity
df_ols = pd.get_dummies(df_ols, columns=['Rating'], drop_first=True)
df_ols = df_ols.drop(columns="Rating_nc-17") # Only 5 nc-17 movies, not considered for analysis
# one-hot encoding month column, dropping first column for same reason
df_ols = pd.get_dummies(df_ols, columns=['Month'], drop_first=True)
# same thing for the emotion column
df_ols = pd.get_dummies(df_ols, columns=['emotion'], drop_first=True)


Checking for multicolinearity

In [775]:
""""
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["Feature"] = df_ols.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)
"""

'"\nfrom statsmodels.stats.outliers_influence import variance_inflation_factor\n\nvif_data = pd.DataFrame()\nvif_data["Feature"] = df_ols.columns\nvif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]\n\nprint(vif_data)\n'

## Defining features and labels for the 4 different models:
Labels: Foreign/domestic percentage for prediction
Predictors: before and after 2010

In [776]:
# Removing Year column, not used as a feature

# Movies before 2010
df_before_2010 = df_ols[df_ols['Year'] < 2010].drop(columns= "Year")

# Movies including and after 2010
df_2010_and_after = df_ols[df_ols['Year'] >= 2010].drop(columns= "Year")

df_ols = df_ols.drop(columns= "Year")


Function to train OLS model

In [777]:
def train_ols_model(dataframe, target_column, drop_columns):
    """
    Trains an OLS regression model.

    Parameters:
        dataframe (pd.DataFrame): The input DataFrame containing features and the target variable.
        target_column (str): The name of the column to be used as the target variable.
        drop_columns (list): List of columns to drop from the DataFrame.

    Returns:
        ols_model (sm.OLS): The trained OLS model.
    """
    # Defining features and labels
    y = dataframe[target_column]
    X = dataframe.drop(columns=drop_columns)

    # Adding a constant column for the OLS model
    X = sm.add_constant(X)

    # Training the OLS model
    feature_names = list(X.columns)
    label_name = target_column

    # Ensure predictors and target are numeric
    X = pd.DataFrame(X, columns=feature_names, dtype=float)
    y = pd.Series(y, name=label_name, dtype=float) # Including the label name

    # Fit the OLS model
    ols_model = sm.OLS(y, X).fit()
    return ols_model

Function to save the feature statistics for later use

In [778]:
def extract_feature_statistics(ols_model, output_csv_path=None):
    """
    Extracts feature statistics from an OLS regression model.

    Parameters:
        ols_model (sm.OLS): The trained OLS regression model.
        output_csv_path (str, optional): Path to save the feature statistics as a CSV file. Defaults to None.

    Returns:
        feature_stats (pd.DataFrame): A DataFrame containing the feature statistics.
    """
    # Extracting feature statistics
    coefficients = ols_model.params  # Coefficients (coef)
    standard_errors = ols_model.bse  # Standard errors (std err)
    t_values = ols_model.tvalues     # t-values (t)
    p_values = ols_model.pvalues     # p-values (P>|t|)

    # Creating a DataFrame
    feature_stats = pd.DataFrame({
        "Feature": coefficients.index,
        "Coefficient": coefficients.values,
        "Std_Error": standard_errors.values,
        "t_value": t_values.values,
        "p_value": p_values.values
    })

    # Saving the DataFrame to a CSV file if the path is provided
    if output_csv_path:
        feature_stats.to_csv(output_csv_path, index=False)

    return feature_stats

### 1st model: Foreign Percentage prediction 

In [779]:
# Defining the target column as well as which columns to drop
target_column = "Foreign_Percentage"
drop_columns = ["Foreign_Percentage", "Domestic_Percentage"]
# Path to save CSV file containing the feature statistics
output_csv_path = "../src/data/Machine_learning_data/feature_stats_Foreign.csv"

# Train the model
ols_model = train_ols_model(df_ols, target_column, drop_columns)

# Extract feature statistics and save to CSV
feature_stats_Foreign = extract_feature_statistics(ols_model, output_csv_path)

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Foreign_Percentage   R-squared:                       0.245
Model:                            OLS   Adj. R-squared:                  0.231
Method:                 Least Squares   F-statistic:                     18.21
Date:                Fri, 20 Dec 2024   Prob (F-statistic):          6.79e-127
Time:                        16:24:24   Log-Likelihood:                -3438.0
No. Observations:                2689   AIC:                             6972.
Df Residuals:                    2641   BIC:                             7255.
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.2410 

### "2nd" model: Domestic Percentage prediction 

In [780]:
# Defining the target column as well as which columns to drop
target_column = "Domestic_Percentage"
drop_columns = ["Foreign_Percentage", "Domestic_Percentage"]
# Path to save CSV file containing the feature statistics
output_csv_path = "../src/data/Machine_learning_data/feature_stats_Domestic.csv"

# Train the model
ols_model = train_ols_model(df_ols, target_column, drop_columns)

# Extract feature statistics and save to CSV
feature_stats_domestic = extract_feature_statistics(ols_model, output_csv_path)

# Displaying the model summary
print(ols_model.summary())

                             OLS Regression Results                            
Dep. Variable:     Domestic_Percentage   R-squared:                       0.245
Model:                             OLS   Adj. R-squared:                  0.231
Method:                  Least Squares   F-statistic:                     18.21
Date:                 Fri, 20 Dec 2024   Prob (F-statistic):          6.79e-127
Time:                         16:24:24   Log-Likelihood:                -3438.0
No. Observations:                 2689   AIC:                             6972.
Df Residuals:                     2641   BIC:                             7255.
Df Model:                           47                                         
Covariance Type:             nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 

Only difference between the two models is the sign of the coeffecients. Subsquent Analysis will have only the foreign_percentage as a target column

### Before 2010, foreign prediction

In [781]:
# Defining the target column as well as which columns to drop
target_column = "Foreign_Percentage"
drop_columns = ["Foreign_Percentage", "Domestic_Percentage"]
# Path to save CSV file containing the feature statistics
output_csv_path = "../src/data/Machine_learning_data/feature_stats_pre_2010.csv"

# Train the model
ols_model = train_ols_model(df_before_2010, target_column, drop_columns)

# Extract feature statistics and save to CSV
feature_stats_pre_2010 = extract_feature_statistics(ols_model, output_csv_path)

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Foreign_Percentage   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.233
Method:                 Least Squares   F-statistic:                     8.957
Date:                Fri, 20 Dec 2024   Prob (F-statistic):           3.94e-51
Time:                        16:24:24   Log-Likelihood:                -1480.8
No. Observations:                1232   AIC:                             3058.
Df Residuals:                    1184   BIC:                             3303.
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.2530 

### Including and after 2010, foreign prediction

In [782]:
# Defining the target column as well as which columns to drop
target_column = "Foreign_Percentage"
drop_columns = ["Foreign_Percentage", "Domestic_Percentage"]
# Path to save CSV file containing the feature statistics
output_csv_path = "../src/data/Machine_learning_data/feature_stats_post_2010.csv"

# Train the model
ols_model = train_ols_model(df_2010_and_after, target_column, drop_columns)

# Extract feature statistics and save to CSV
feature_stats_post_2010 = extract_feature_statistics(ols_model, output_csv_path)

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Foreign_Percentage   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                  0.241
Method:                 Least Squares   F-statistic:                     10.84
Date:                Fri, 20 Dec 2024   Prob (F-statistic):           1.48e-65
Time:                        16:24:24   Log-Likelihood:                -1873.4
No. Observations:                1457   AIC:                             3843.
Df Residuals:                    1409   BIC:                             4096.
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.3718 

## Comparing the statistically significant coefficients

### Filtering significant features

In [783]:
feature_stats_pre_2010.head(5)

,Feature,Coefficient,Std_Error,t_value,p_value
0,const,-0.253036,0.214662,-1.178766,2.387284e-01
1,Budget(USD)_Inflated,0.214164,0.038253,5.598661,2.682997e-08
2,Runtime(mins),0.009215,0.033120,0.278235,7.808809e-01
3,Audience_Score,-0.077959,0.034704,-2.246367,2.486422e-02
4,Critics_Score,0.143675,0.034302,4.188542,3.015804e-05


Defining funciton that removes all the features with a P-value > 0.05

In [784]:
def filter_significant_features(feature_stats, p_value_threshold=0.05):

    # Filter the DataFrame based on the p-value threshold
    significant_features = feature_stats[feature_stats["p_value"] <= p_value_threshold]
    
    return significant_features

In [785]:
feature_stats_Foreign  = filter_significant_features(feature_stats_Foreign)
feature_stats_pre_2010 = filter_significant_features(feature_stats_pre_2010)
feature_stats_post_2010 = filter_significant_features(feature_stats_post_2010)


### Plotting the coeffecients for the statistically significant features

In [786]:
def plot_feature_coefficients(dataframe, title, output_path = None):
    """
    Plots a bar chart for feature coefficients with predefined parameters.

    Parameters:
        dataframe (pd.DataFrame): DataFrame containing the feature statistics to plot.
        title (str): Title of the bar chart.
        Optional: output_path (str): Path to save the HTML file. If None, the plot is not saved.

    Returns:
        fig: The Plotly bar chart figure.
    """
    # Aranges the features in descending order base on their coeffecients
    dataframe_sorted = dataframe.sort_values(by="Coefficient", ascending=False)

    # Create the bar plot
    fig = px.bar(
        dataframe_sorted,
        x='Feature',
        y='Coefficient',
        hover_data=['Std_Error', 't_value', 'p_value'],
        color='Coefficient',
        labels={'Feature': 'Feature Names', 'Coefficient': 'Coefficient Value'},
        title=title,
        height=400
    )

    # Centering the title and changing the font

    fig.update_layout(
        title={
            'text': title,
            'y': 0.95,  # Vertical alignment of the title
            'x': 0.5,   # Horizontal alignment of the title
            'xanchor': 'center',
            'yanchor': 'top'
        },
        font=dict(
            family="Futura",  # Change to your preferred font
            size=15,                     # Title font size
            color="black"                # Title font color
        ),
        title_font=dict(
            size=20   
        )
    )

    # Save the plot as html file if output_path provided
    if output_path:
        fig.write_html(output_path)
        print(f"Plot saved to {output_path}")
    else:
        print("Plot not saved.")

    return fig

In [787]:
plot_feature_coefficients(feature_stats_Foreign, "OLS Coeffecients 2000-2019", "../_includes/OLS_features_all_years.html")


Plot saved to ../_includes/OLS_features_all_years.html


In [788]:
plot_feature_coefficients(feature_stats_pre_2010, "OLS Coeffecients 2000-2009", "../_includes/OLS_features_pre_2010.html")

Plot saved to ../_includes/OLS_features_pre_2010.html


In [789]:
plot_feature_coefficients(feature_stats_post_2010, "OLS Coeffecients 2010-2019", "../_includes/OLS_features_post_2010.html")

Plot saved to ../_includes/OLS_features_post_2010.html
